In [1]:
%matplotlib inline
from matplotlib import pylab as plt
from ipywidgets import interact
import ipywidgets as widgets
import matplotlib as mpl
import numpy as np

In [2]:
from geonotebook.wrappers import RasterDataCollection
import os

In [3]:
DATA_DIR="/data/kotfic/NEX/golden_tile_layer/WELD/golden_tiles/geotiff/NBAR/"

In [4]:
def sort_NBAR(a, b):
  am, ay = int(a.split(".")[2][-2:]), int(a.split(".")[3])
  bm, by = int(b.split(".")[2][-2:]), int(b.split(".")[3])

  if ay < by:
    return -1
  elif ay > by:
    return 1
  elif by == ay:
    if am < bm:
      return -1
    elif am > bm:
      return 1
    else:
      return 0
  

PATHS = [DATA_DIR + p for p in sorted(os.listdir(DATA_DIR), sort_NBAR)]

In [5]:
PATHS

['/data/kotfic/NEX/golden_tile_layer/WELD/golden_tiles/geotiff/NBAR/L57.Globe.month12.2008.hh09vv04.h6v1.doy339to366.NBAR.v3.0.tiff',
 '/data/kotfic/NEX/golden_tile_layer/WELD/golden_tiles/geotiff/NBAR/L57.Globe.month01.2009.hh09vv04.h6v1.doy005to031.NBAR.v3.0.tiff',
 '/data/kotfic/NEX/golden_tile_layer/WELD/golden_tiles/geotiff/NBAR/L57.Globe.month02.2009.hh09vv04.h6v1.doy032to055.NBAR.v3.0.tiff',
 '/data/kotfic/NEX/golden_tile_layer/WELD/golden_tiles/geotiff/NBAR/L57.Globe.month03.2009.hh09vv04.h6v1.doy060to087.NBAR.v3.0.tiff',
 '/data/kotfic/NEX/golden_tile_layer/WELD/golden_tiles/geotiff/NBAR/L57.Globe.month04.2009.hh09vv04.h6v1.doy093to120.NBAR.v3.0.tiff',
 '/data/kotfic/NEX/golden_tile_layer/WELD/golden_tiles/geotiff/NBAR/L57.Globe.month05.2009.hh09vv04.h6v1.doy124to151.NBAR.v3.0.tiff',
 '/data/kotfic/NEX/golden_tile_layer/WELD/golden_tiles/geotiff/NBAR/L57.Globe.month06.2009.hh09vv04.h6v1.doy152to181.NBAR.v3.0.tiff',
 '/data/kotfic/NEX/golden_tile_layer/WELD/golden_tiles/geotiff

In [6]:
rdc = RasterDataCollection(PATHS)

In [7]:
M.add_layer(rdc[:,[1,2,3]], 'NBAR', opacity=0.8)

In [14]:
M.layers["NBAR"].next()

In [9]:
M.layers["NBAR"].prev()

In [15]:
M.layers["NBAR"].seek(0)

In [16]:
len(M.layers["NBAR"].data)

36

In [17]:
def render_timeseries(idx=0):
  M.layers["NBAR"].seek(idx)
  
interact(render_timeseries, idx=(0, len(M.layers["NBAR"].data) - 1))

In [18]:
M.layers["NBAR"].seek(1)

In [19]:
M.remove_layer(M.layers[-1].name)

In [20]:
M.add_layer(rdc[:,[4]], 'NBAR_NDVI', opacity=0.8)

In [29]:
M.remove_layer("NBAR_NDVI")

In [22]:
def ndvi_colormap(numcolors=11, name='custom_div_cmap', 
                  mincol='blue', midcol='beige', maxcol='green'):
  return mpl.colors.LinearSegmentedColormap.from_list(
    name=name, colors=[mincol, midcol, maxcol], N=numcolors)

cmap = ndvi_colormap()


In [23]:
colormap = [{"color": mpl.colors.rgb2hex(cmap(i)), "quantity": v }
 for i,v in zip(range(cmap.N),np.linspace(min(rdc[:,4].min), max(rdc[:,4].max), cmap.N))]

In [24]:
colormap

[{'color': u'#0000ff', 'quantity': -1.0},
 {'color': u'#3131f8', 'quantity': -0.85065351524498001},
 {'color': u'#6262f1', 'quantity': -0.70130703048996001},
 {'color': u'#9393ea', 'quantity': -0.55196054573494002},
 {'color': u'#c4c4e3', 'quantity': -0.40261406097992003},
 {'color': u'#f5f5dc', 'quantity': -0.25326757622490004},
 {'color': u'#c4deb0', 'quantity': -0.10392109146988004},
 {'color': u'#93c684', 'quantity': 0.045425393285139837},
 {'color': u'#62af58', 'quantity': 0.19477187804015994},
 {'color': u'#31972c', 'quantity': 0.34411836279518004},
 {'color': u'#008000', 'quantity': 0.49346484755019998}]

In [25]:
M.add_layer(rdc[:,4], 'NBAR_NDVI', colormap=colormap, opacity=0.8)

In [27]:
M.layers['NBAR_NDVI'].next()

In [31]:
M.layers

GeonotebookStack([<TimeSeriesLayer('NBAR')>, <TimeSeriesLayer('NBAR_NDVI')>])